In [ ]:
from pathlib import Path

from prepare_data import get_dataframe

crud = get_dataframe(
    Path("../../Static Analysis/impl-crud/crud_chidamber-kemerer.csv")
)
cqrs = get_dataframe(
    Path("../../Static Analysis/impl-es-cqrs/es-cqrs-chidamber-kemerer.csv")
)

In [ ]:
labels = crud.columns[1:].tolist()
crud_data_matrix = crud.iloc[:, 1:].values
cqrs_data_matrix = cqrs.iloc[:, 1:].values

In [ ]:
import numpy as np

combined_data = np.vstack([crud_data_matrix, cqrs_data_matrix])
max_vals = np.nanmax(combined_data, axis=0)
max_vals[max_vals == 0] = 1

# 2. Normalize both datasets
crud_norm = crud_data_matrix / max_vals
cqrs_norm = cqrs_data_matrix / max_vals

# calculate the Mean for each metric
# result is a 1D array of 6 values (one for each metric)
crud_med = np.median(crud_norm, axis=0)
cqrs_med = np.median(cqrs_norm, axis=0)

crud_m = np.mean(crud_norm, axis=0)
cqrs_m = np.mean(cqrs_norm, axis=0)

In [ ]:
import matplotlib.pyplot as plt
from spider_plot import radar_factory

plt.style.use('default')

N = len(labels)
theta = radar_factory(N, frame='polygon')

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(projection='radar'))

ax.plot(theta, crud_m + 0.002, color='b', label='CRUD Architecture', linewidth=2)
ax.fill(theta, crud_m, facecolor='b', alpha=0.25)

ax.plot(theta, cqrs_m, color='r', label='ES-CQRS Architecture', linewidth=2)
ax.fill(theta, cqrs_m, facecolor='r', alpha=0.25)

ax.set_varlabels(labels)
ax.set_ylim(0, .15)
ax.set_rgrids([0.05, 0.1, 0.15, 0.2, 0.25])
ax.set_title("Architecture Comparison: Mean Chidamber-Kemerer Metrics", position=(0.5, 1.1), ha='center')
ax.legend()

plt.show()